Capstone Final Project

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    scipy-1.4.1                |   py36h921218d_0        18.9 MB  conda-forge
    beautifulsoup4-4.8.2       |           py36_0         157 KB  conda-forg

Import needed libraries

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.1

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          91 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.50-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.50   | 3

Load the data set

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


Pull out the features we want to see

In [4]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [5]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


Make sure it's working... 

In [6]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


Look just at Manhattan data

In [7]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [8]:
manhattan_data.shape

(40, 4)

Create a map of New York

In [9]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Link the map to Foursquare

In [11]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

Load my credentials

In [12]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'FDCXNN21TLUUE3NFHW15V15BSIJXZ5WB2QXCEIKIOC35LZAE'
CLIENT_SECRET = 'SJG53ZXNSDYKVTYPUE4HMLA1IR45PBFQ4FLBYXR423N5BGAE'
VERSION = '20181020'

Apply the catagory ID for groceries

In [13]:
#https://developer.foursquare.com/docs/resources/categories
#Grocery Stores = 4bf58dd8d48988d118951735
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_grocery = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d118951735')
newyork_venues_grocery.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,ALDI,40.877836,-73.904656,Supermarket
1,Marble Hill,40.876551,-73.91066,Garden Gourmet Market,40.881350,-73.903389,Gourmet Shop
2,Marble Hill,40.876551,-73.91066,Ben's Market Basket,40.878591,-73.917486,Grocery Store
3,Marble Hill,40.876551,-73.91066,Bravo Supermarkets,40.878409,-73.904198,Grocery Store
4,Marble Hill,40.876551,-73.91066,Glatt Shop,40.886317,-73.909678,Grocery Store


How many are there?

In [14]:
newyork_venues_grocery.shape

(1248, 7)

Add grocery to existing map

In [15]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

Now create the map with existing grocery locations on it

In [16]:
map_newyork_grocery = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_grocery, 'red', map_newyork_grocery)

map_newyork_grocery

In [17]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0  

In [19]:
manhattan_grouped = newyork_venues_grocery.groupby('Neighborhood').count()
manhattan_grouped
#print('There are {} uniques categories.'.format(len(newyork_venues_grocery['Venue Category'].unique())))

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,16,16,16,16,16,16
Carnegie Hill,31,31,31,31,31,31
Central Harlem,25,25,25,25,25,25
Chelsea,33,33,33,33,33,33
Chinatown,49,49,49,49,49,49
Civic Center,50,50,50,50,50,50
Clinton,25,25,25,25,25,25
East Harlem,28,28,28,28,28,28
East Village,50,50,50,50,50,50


In [58]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_grocery[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_grocery['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Bakery,Buffet,Butcher,Chinese Restaurant,Convenience Store,Deli / Bodega,Department Store,Discount Store,Dumpling Restaurant,Farmers Market,Food & Drink Shop,Fruit & Vegetable Store,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Health Food Store,Market,Mexican Restaurant,Nutritionist,Organic Grocery,Pharmacy,Restaurant,Sandwich Place,Supermarket,Vegetarian / Vegan Restaurant
0,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [59]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Bakery,Buffet,Butcher,Chinese Restaurant,Convenience Store,Deli / Bodega,Department Store,Discount Store,Dumpling Restaurant,Farmers Market,Food & Drink Shop,Fruit & Vegetable Store,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Health Food Store,Market,Mexican Restaurant,Nutritionist,Organic Grocery,Pharmacy,Restaurant,Sandwich Place,Supermarket,Vegetarian / Vegan Restaurant
0,Battery Park City,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000
1,Carnegie Hill,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032258,0.000000,0.000000,0.000000,0.032258,0.000000,0.774194,0.000000,0.032258,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.096774,0.000000
2,Central Harlem,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.840000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.120000,0.000000
3,Chelsea,0.000000,0.000000,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.030303,0.030303,0.000000,0.757576,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000
4,Chinatown,0.000000,0.000000,0.040816,0.000000,0.000000,0.020408,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.040816,0.000000,0.795918,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.081633,0.000000
5,Civic Center,0.000000,0.000000,0.040000,0.020000,0.000000,0.040000,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.040000,0.000000,0.740000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000
6,Clinton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.080000,0.000000,0.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.120000,0.000000
7,East Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.928571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000
8,East Village,0.000000,0.000000,0.000000,0.000000,0.020000,0.060000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.740000,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.000000,0.000000,0.080000,0.020000
9,Financial District,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.764706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117647,0.000000


In [60]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
num_top_venues = 4

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Battery Park City,Grocery Store,Supermarket,Deli / Bodega,Food & Drink Shop
1,Carnegie Hill,Grocery Store,Supermarket,Butcher,Market
2,Central Harlem,Grocery Store,Supermarket,Butcher,Vegetarian / Vegan Restaurant
3,Chelsea,Grocery Store,Supermarket,Gluten-free Restaurant,Food & Drink Shop
4,Chinatown,Grocery Store,Supermarket,Butcher,Gourmet Shop
5,Civic Center,Grocery Store,Supermarket,Butcher,Deli / Bodega
6,Clinton,Grocery Store,Supermarket,Gourmet Shop,Vegetarian / Vegan Restaurant
7,East Harlem,Grocery Store,Deli / Bodega,Supermarket,Vegetarian / Vegan Restaurant
8,East Village,Grocery Store,Supermarket,Deli / Bodega,Vegetarian / Vegan Restaurant
9,Financial District,Grocery Store,Supermarket,Deli / Bodega,Food & Drink Shop


In [71]:
# set number of clusters
kclusters = 4

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 0, 2, 0, 0], dtype=int32)

In [72]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,3,Grocery Store,Supermarket,Gourmet Shop,Vegetarian / Vegan Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,0,Grocery Store,Supermarket,Butcher,Gourmet Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Grocery Store,Convenience Store,Vegetarian / Vegan Restaurant,Fruit & Vegetable Store
3,Manhattan,Inwood,40.867684,-73.921210,3,Grocery Store,Supermarket,Discount Store,Vegetarian / Vegan Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Grocery Store,Supermarket,Vegetarian / Vegan Restaurant,Fruit & Vegetable Store


In [65]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [73]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
1,Chinatown,0,Grocery Store,Supermarket,Butcher,Gourmet Shop
8,Upper East Side,0,Grocery Store,Supermarket,Farmers Market,Butcher
9,Yorkville,0,Grocery Store,Supermarket,Butcher,Market
10,Lenox Hill,0,Grocery Store,Supermarket,Farmers Market,Butcher
12,Upper West Side,0,Grocery Store,Supermarket,Deli / Bodega,Vegetarian / Vegan Restaurant
13,Lincoln Square,0,Grocery Store,Supermarket,Gourmet Shop,Bakery
14,Clinton,0,Grocery Store,Supermarket,Gourmet Shop,Vegetarian / Vegan Restaurant
15,Midtown,0,Grocery Store,Supermarket,Deli / Bodega,Nutritionist
16,Murray Hill,0,Grocery Store,Deli / Bodega,Supermarket,Restaurant
17,Chelsea,0,Grocery Store,Supermarket,Gluten-free Restaurant,Food & Drink Shop


In [74]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
11,Roosevelt Island,1,Grocery Store,Deli / Bodega,Supermarket,Farmers Market
24,West Village,1,Grocery Store,Supermarket,Food & Drink Shop,Deli / Bodega
35,Turtle Bay,1,Grocery Store,Deli / Bodega,Supermarket,Pharmacy
36,Tudor City,1,Grocery Store,Deli / Bodega,Supermarket,Pharmacy


In [75]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
2,Washington Heights,2,Grocery Store,Convenience Store,Vegetarian / Vegan Restaurant,Fruit & Vegetable Store
4,Hamilton Heights,2,Grocery Store,Supermarket,Vegetarian / Vegan Restaurant,Fruit & Vegetable Store
5,Manhattanville,2,Grocery Store,Butcher,Supermarket,Vegetarian / Vegan Restaurant
6,Central Harlem,2,Grocery Store,Supermarket,Butcher,Vegetarian / Vegan Restaurant
7,East Harlem,2,Grocery Store,Deli / Bodega,Supermarket,Vegetarian / Vegan Restaurant
26,Morningside Heights,2,Grocery Store,Supermarket,Sandwich Place,Butcher
31,Noho,2,Grocery Store,Supermarket,Organic Grocery,Convenience Store


In [76]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
0,Marble Hill,3,Grocery Store,Supermarket,Gourmet Shop,Vegetarian / Vegan Restaurant
3,Inwood,3,Grocery Store,Supermarket,Discount Store,Vegetarian / Vegan Restaurant
39,Hudson Yards,3,Grocery Store,Supermarket,Gourmet Shop,Gluten-free Restaurant


In [77]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(4, manhattan_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue
